In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# Hyper-parameters
batch_size = 128


mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean,std)]
)

# transform = transforms.Compose(
#     [transforms.ToTensor()]
# )

train_data = torchvision.datasets.CIFAR100(
    root="./data",
    train=True,
    transform=transform,
    download=True
)

test_data = torchvision.datasets.CIFAR100(
    root="./data",
    train=False,
    transform=transform,
    download=True
)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=True)


100%|██████████| 169001437/169001437 [00:03<00:00, 47390347.88it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 16, 5)
        self.conv3 = nn.Conv2d(16, 24, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 100)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x,1)
        # x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
def train_step(model ,epoch, loss_function, optimizer, loss_train_epoch):
    # model.train()
    epoch_loss = []
    loop = tqdm(enumerate(train_loader))
    loop.set_description(f"Epoch {epoch}")
    for batch_idx, (data, target) in loop:

        data, target = data.to(device), target.to(device)

        output = model(data)
        loss = loss_function(output, target)
        epoch_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loop.set_postfix(
            {
                "Iter": "[{}/{}]".format(
                    batch_idx * len(data), len(train_loader.dataset)
                ),
                "Epoch_Loss": sum(epoch_loss) / len(epoch_loss),
                "Batch_Loss": loss.item(),
            }
        )
    loss_train_epoch.append(sum(epoch_loss) / len(epoch_loss))

    return loss_train_epoch


def test(model, name, loss_function, best_auc, loss_test_epoch, epoch_auc_history):
    model.eval()
    test_loss_list = []
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss_list.append(loss_function(output, target).item())
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = sum(test_loss_list) / len(test_loss_list)
    loss_test_epoch.append(test_loss)
    epoch_auc_history.append(correct / 100)

    if correct > best_auc:
        best_auc = correct
        torch.save(model.state_dict(), f"pretrained/{name}/{name}.pth")

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

    print("Best Accuracy :", best_auc)

    return best_auc, loss_test_epoch, epoch_auc_history

In [6]:
learning_rate = 0.001
num_epochs = 25
loss_train_epoch = []
epoch_auc_history = []
loss_test_epoch = []
best_auc = 0

model = ConvNet().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

for epoch in range(1, num_epochs):
    loss_train_epoch = train_step(model, epoch, loss_function, optimizer, loss_train_epoch)

    best_auc, loss_test_epoch, epoch_auc_history = test(model, "conv", loss_function, best_auc, loss_test_epoch, epoch_auc_history)

    scheduler.step()


Epoch 1: : 391it [00:18, 20.83it/s, Iter=[31200/50000], Epoch_Loss=3.81, Batch_Loss=3.32]



Test set: Average loss: 3.4272, Accuracy: 1829/10000 (18%)

Best Accuracy : 1829


Epoch 2: : 391it [00:17, 22.58it/s, Iter=[31200/50000], Epoch_Loss=3.22, Batch_Loss=2.88]



Test set: Average loss: 3.0989, Accuracy: 2462/10000 (25%)

Best Accuracy : 2462


Epoch 3: : 391it [00:17, 21.89it/s, Iter=[31200/50000], Epoch_Loss=2.93, Batch_Loss=3.01]


...


Test set: Average loss: 4.0823, Accuracy: 3318/10000 (33%)

Best Accuracy : 3507


Epoch 23: : 391it [00:16, 23.39it/s, Iter=[31200/50000], Epoch_Loss=0.715, Batch_Loss=0.825]



Test set: Average loss: 4.2279, Accuracy: 3271/10000 (33%)

Best Accuracy : 3507


Epoch 24: : 391it [00:17, 22.61it/s, Iter=[31200/50000], Epoch_Loss=0.672, Batch_Loss=0.541]



Test set: Average loss: 4.3471, Accuracy: 3260/10000 (33%)

Best Accuracy : 3507


In [6]:
path = "/content/pretrained/conv/conv.pth"
loaded_model = ConvNet().to(device)
loaded_model.load_state_dict(torch.load(path))
loaded_model.eval()

with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)
    total_inference_time = 0

    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = loaded_model(images)

      _, predicted = torch.max(outputs,1) #Return the max value and index of all the elements in the input tensor i.e., highest probablity
      n_correct += (predicted == labels).sum().item()

    acc = 100.0 * (n_correct/n_samples)
    print(f'Accuracy of the model: {acc:.4f}%')


Accuracy of the model: 35.0700%


In [7]:
model_prt = torchvision.models.resnet152(weights='DEFAULT')
model_prt

Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:02<00:00, 118MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
for param in model_prt.parameters():
    param.requires_grad = False

num_ftrs = model_prt.fc.in_features
model_prt.fc = nn.Linear(num_ftrs, 100)

model_prt.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
learning_rate = 0.001
num_epochs = 200
loss_train_epoch = []
epoch_auc_history = []
loss_test_epoch = []
best_auc = 0

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_prt.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)

for epoch in range(1, num_epochs):
    loss_train_epoch = train_step(model_prt, epoch, loss_function, optimizer, loss_train_epoch)

    best_auc, loss_test_epoch, epoch_auc_history = test(model_prt, "resnet", loss_function, best_auc, loss_test_epoch, epoch_auc_history)

    scheduler.step()


Epoch 1: : 391it [01:11,  5.48it/s, Iter=[31200/50000], Epoch_Loss=2.46, Batch_Loss=2.04]



Test set: Average loss: 3.9090, Accuracy: 4951/10000 (50%)

Best Accuracy : 4951


Epoch 2: : 391it [01:13,  5.33it/s, Iter=[31200/50000], Epoch_Loss=2.36, Batch_Loss=1.79]



Test set: Average loss: 1.9231, Accuracy: 4828/10000 (48%)

Best Accuracy : 4951


Epoch 3: : 391it [01:14,  5.28it/s, Iter=[31200/50000], Epoch_Loss=1.61, Batch_Loss=1.78]


...

Epoch 152: : 391it [01:12,  5.37it/s, Iter=[31200/50000], Epoch_Loss=0.000252, Batch_Loss=1.19e-8]



Test set: Average loss: 6.5720, Accuracy: 5966/10000 (60%)

Best Accuracy : 5983


Epoch 153: : 391it [01:12,  5.37it/s, Iter=[31200/50000], Epoch_Loss=0.000252, Batch_Loss=1.19e-8]



Test set: Average loss: 6.5594, Accuracy: 5966/10000 (60%)

Best Accuracy : 5983


Epoch 154: : 214it [00:39,  5.45it/s, Iter=[27264/50000], Epoch_Loss=0.000255, Batch_Loss=9.31e-10]

In [13]:
path = "/content/pretrained/resnet/resnet.pth"
loaded_model_prt = model_prt.to(device)
loaded_model_prt.load_state_dict(torch.load(path))
loaded_model_prt.eval()

with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)
    total_inference_time = 0

    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = loaded_model_prt(images)

      _, predicted = torch.max(outputs,1) #Return the max value and index of all the elements in the input tensor i.e., highest probablity
      n_correct += (predicted == labels).sum().item()

    acc = 100.0 * (n_correct/n_samples)
    print(f'Accuracy of the model: {acc:.4f}%')

Accuracy of the model: 59.8300%
